In [ ]:

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch
import pandas as pd
import scanpy as sc
import numpy as np
from torch.utils.data import TensorDataset
train_data=pd.read_csv("test_data.csv")
#print(train_data)
train_data=train_data.T
train_data.to_csv("testT_data.csv")
data=train_data.values
data_values=data.T  #进行转置？
#print(train_data)
adata=sc.read_csv("testT_data.csv")#应该进行装置
adata=adata[1:201]
print(type(data))
a,gene_num=adata.shape
sc.pp.filter_cells(adata, min_counts=1)
adata.raw = adata.copy()
adata.obs['size_factors'] = adata.obs.n_counts / np.median(adata.obs.n_counts)
sc.pp.log1p(adata)
sc.pp.scale(adata)

进行预处理

In [ ]:
#编码方式
class seq_dataset(Dataset):
    def __init__(self,load_data):
        self.data = load_data
    def __getitem__(self, idx):
        return self.data[idx]
    def __len__(self,):
        return len(self.data)
RNA=seq_dataset(data_values)
train_RNA_dataloader=DataLoader(RNA,batch_size=32,shuffle=True)
count_mtx = torch.tensor(adata.X)
count_mtx_raw = torch.tensor(adata.raw.X)
size_factor = torch.tensor(adata.obs.size_factors.values)
dataset = TensorDataset(count_mtx, size_factor,count_mtx_raw)
#print(dataset)
train1_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

网络处理

In [ ]:
def weights_init(mat):
    if type(mat) == nn.Linear:
        torch.nn.init.xavier_uniform_(mat.weight,gain=3)
        print("Weight:",mat.weight)
        mat.bias.data.fill_(0.1)

In [ ]:
class mean_activation(nn.Module):
    def __init__(self):
        super(mean_activation,self).__init__()
    def forward(self,input):
        return torch.exp(input)

class dispersion_activiation(nn.Module):
    def __init__(self):
        super(dispersion_activiation,self).__init__()
    def forward(self,input):
        return torch.exp(input)
    

In [ ]:
class auto_encoder(nn.Module):
    def __init__( self,gene_num,hidden_dim):
        super(auto_encoder,self).__init__()
        self.gene_num = gene_num
        print("gene_num: " ,gene_num)

        self.autoencoder_stack=nn.Sequential(
            nn.Linear(gene_num,hidden_dim[0]),
            nn.BatchNorm1d(hidden_dim[0],eps=1e-2,affine=False), 
            nn.ReLU(),
            nn.Linear(hidden_dim[0],hidden_dim[1]),
            nn.BatchNorm1d(hidden_dim[1],eps=1e-2,affine=False),
            nn.ReLU(),
            nn.Linear(hidden_dim[1],hidden_dim[2]),
            nn.BatchNorm1d(hidden_dim[2],eps=1e-2,affine=False),
            nn.ReLU(),
        )
        self.mean=nn.Sequential(
            nn.Linear(hidden_dim[2],gene_num),
            mean_activation()
        )
        self.pi=nn.Sequential(
            nn.Linear(hidden_dim[2],gene_num),
            nn.Sigmoid()
        )
        self.theta=nn.Sequential(
            nn.Linear(hidden_dim[2],gene_num),
            dispersion_activiation()
        )

    def forward(self, input, size_factor):
        size_factor = size_factor.view(-1, 1)#?
        size_factor = size_factor.repeat(1, self.gene_num)
        #ae_out = input * size_factor
        ae_out = self.autoencoder_stack(input) #?
        #print("AE:  ", ae_out)
        mean = self.mean(ae_out) * size_factor
        #print("MEAN:  ", mean)
        theta = self.theta(ae_out)
        pi = self.pi(ae_out)
        return mean, theta, pi

        


In [ ]:
def _nan2inf(x):
    return torch.where(torch.isnan(x), torch.zeros_like(x) + np.inf, x)
class NB(torch.nn.Module):
    def __init__(self, scale_factor=1.0):
        super(NB, self).__init__()
        self.eps = 1e-10
        self.scale_factor = scale_factor

    def forward(self, theta, y_true, y_pred, mean=True):
        y_pred = y_pred * self.scale_factor
        t1 = torch.lgamma(theta + self.eps) + torch.lgamma(y_true + 1.0) - torch.lgamma(y_true + theta + self.eps)
        t2 = (theta + y_true) * torch.log(1.0 + (y_pred / (theta + self.eps))) + (
                y_true * (torch.log(theta + self.eps) - torch.log(y_pred + self.eps)))

        final = t1 + t2
        final = _nan2inf(final)

        if mean:
            final = torch.mean(final)

        return final
class ZINB(NB):
    def __init__(self, scale_factor=1.0, ridge_lambda=0.0):
        super().__init__(scale_factor=scale_factor)
        self.ridge_lambda = ridge_lambda
        self.scale_factor = scale_factor

    def forward(self, pi, theta, y_true, y_pred, mean=True):
        nb_case = super().forward(theta, y_true, y_pred, mean=False) - torch.log(1.0 - pi + self.eps)

        y_pred = y_pred * self.scale_factor
        # theta = torch.minimum(theta, torch.tensor(1e6))

        zero_nb = torch.pow(theta / (theta + y_pred + self.eps), theta)
        zero_case = -torch.log(pi + ((1.0 - pi) * zero_nb) + self.eps)
        result = torch.where(torch.lt(y_true, 1e-8), zero_case, nb_case)

        if self.ridge_lambda > 0:
            ridge = self.ridge_lambda * torch.square(pi)
            result += ridge

        if mean:
            result = torch.mean(result)

        result = _nan2inf(result)

        return result

In [ ]:
learning_rate = 1e-3
batch_size = 32
epochs = 180
grad_clip_val = 5.0

In [ ]:

#gene_num=count_mtx_raw.shape[1]
model = auto_encoder(gene_num=gene_num, hidden_dim=(64, 32, 64))
model.apply(weights_init)#不完整

In [ ]:
loss_fn = ZINB()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, epochs / 2)

训练

In [ ]:
def train(dataloader,model,loss,optimizer):
    #size_data=len(dataloader.dataset)
    for batch, (X, size, y) in enumerate(dataloader):
        optimizer.zero_grad()
    # Predict, and calculate loss
        mean, theta, pi = model(y, size)
        #loss_val = loss(mean, theta, pi, y)
        loss_val= loss (pi,theta,y,mean,mean=True)
        loss_val.backward()
        nn.utils.clip_grad_norm_(model.parameters(), grad_clip_val)
        optimizer.step()
        if batch % 50 == 0:
            #loss, curr = loss_val, batch * len(X)
            print(f"loss: {loss_val}")
        

In [ ]:
for iter in np.arange(epochs):
    print(f"Epoch {iter}\n-------------------------------")
    train(train1_dataloader,model,loss_fn,optimizer)
    scheduler.step()

In [ ]:
mean, theta, pi = model(count_mtx_raw, size_factor)
print(mean)
mean_value=mean.detach().numpy()
#print(theta)
#print(pi)
#print(mean.shape,train_data.shape)

结果检验

In [ ]:
import copy
def impute_zero(mean, previous_data):
    imputed=previous_data.copy()
    print(imputed)
    for item in np.arange(imputed.shape[0]):
        for ele in np.arange(imputed.shape[1]):
            if imputed[item][ele]==0:
                imputed[item][ele]=mean[item][ele]
    return imputed
    
    

In [ ]:
imputed_res=impute_zero(mean,data)
print(imputed_res)


In [ ]:
def previous_diff(previous,file_path):
    true_data=pd.read_csv(file_path)
    true_data=np.array(true_data)
    true_data=true_data.T
    dist=0
    for item in np.arange(previous.shape[0]):
        for ele in np.arange(previous.shape[1]):
            if previous[item][ele]==0:
                res=true_data[item][ele]
                dist=dist+res
    print("Previous Value Sum: ",dist)
def calculate_distance(mat,file_path,imputed):
    distance = 0;
    count_pre,count_after=0,0
    true_data=pd.read_csv(file_path)
    true_data=np.array(true_data)
    true_data=true_data.T
    print(data.shape,true_data.shape)
    for item in np.arange(mat.shape[0]):
        for ele in np.arange(mat.shape[1]):
            if mat[item][ele]==0:
                count_pre+=1
                if np.round(imputed[item][ele])==0:
                    count_after+=1
                res=np.round(imputed[item][ele]-true_data[item][ele])
                #print(mat[item][ele]-true_data[item][ele])
                distance+=np.abs(res)
    print("Dropout Previous :",count_pre/(mat.shape[0]*mat.shape[1]))
    print("Dropout Imputed :",count_after/(mat.shape[0]*mat.shape[1]))
    print("Imputed Value Sum:",distance)
    return distance
dist=calculate_distance(data,"test_truedata.csv",mean_value)
previous_diff(data,"test_truedata.csv")

对大数据集进行计算

In [ ]:
data_test=pd.read_csv("data.csv")
test_data=data_test.T
test_data.to_csv("data4test.csv")
finaldata=test_data.values
data_values=finaldata.T  #进行转置？
data2test=sc.read_csv("data4test.csv")
data2test=data2test[1:data2test.shape[0]]
print(data2test.shape)



In [ ]:
sc.pp.filter_cells(data2test, min_counts=1)
data2test.raw = data2test.copy()
data2test.obs['size_factors'] = data2test.obs.n_counts / np.median(data2test.obs.n_counts)
sc.pp.log1p(data2test)
sc.pp.scale(data2test)
count_mtx_raw_1 = torch.tensor(data2test.raw.X)
size_factor_1 = torch.tensor(data2test.obs.size_factors.values)
means, theta, pi = model(count_mtx_raw_1, size_factor_1)
mean_value_final=means.detach().numpy()
print(mean_value_final.shape,test_data.shape)



In [ ]:
results=impute_zero(mean_value_final,finaldata)
final_res=pd.DataFrame(results.T)
final_res.to_csv("results.csv",index=0,header=0)
